# Preliminary Settings

In [1]:
import os

# switch working directory to working directory of main repository 
os.chdir("./..")

In [2]:
from codes import *

reorganize_data()

Data already reorganized!


# Load Data

In [3]:
(X_train, y_train), (X_valid, y_valid), (X_test, y_test) = load_data();

# Split Channels

In [4]:
# image preprocessing step
image_processor = image_preprocessing()

# preprocess train data
_ = image_processor.split_channels(X_train)
_,_ = image_processor.ROI()
Xn_train = image_processor.image_normalize(option="ROI_on_whole")

# preprocess valid data
_ = image_processor.split_channels(X_valid)
_,_ = image_processor.ROI()
Xn_valid = image_processor.image_normalize(option="ROI_on_whole")

# preprocess train data
_ = image_processor.split_channels(X_test)
_,_ = image_processor.ROI()
Xn_test = image_processor.image_normalize(option="ROI_on_whole")

# Binary mask on Red Channel

# Crop ROI on Green Channel

# Feature Extraction

## Shift-Invariant Feature Transform (SIFT) for Bag of Feature (BoF)

## Local Discriminant Basis (LDB) using Wavelets Transform

## Stationary Wavelet Transform (SWT) Based Feature Extractor

## Haralick texture features based on GLCM (gray level co-occurrence matrix)

## Speeded-up Robust Features (SURF) for Bag of Feature (BoF)

## Wavelets Scattering

# Dimension Reduction

# Model Fitting

## SIFT Based Model

## LDB Based Model

## SWT Based Model

## Haralick Based Model

## SURF Based Model

## Wavelets Scattering Based Model